In [15]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSnwd-zccEOQbpNWdItUG0qXND5rPVFbowZINjugi15TdWgqiy3A8eMRhbmSMBiRhHt1Qsry3E8tKY8/pub?output=csv'
df = pd.read_csv(url)
df



,Program,Description,Command/ Code,Comments
0,API,Batch Vs API,NaN,"Batch processing is simplier, and more heavily..."
1,Debian,apt-get,NaN,Command-line-centric and might have a steeper ...
2,Debian,apt,NaN,"User-friendly and intuitive, providing a simpl..."
3,Docker,Run Image,docker run -it NAME /bin/sh,NaN
4,Docker,Check Open Containers,docker ps -a,NaN
...,...,...,...,...
187,Anaconda,Delete Environment,conda env remove -n py39,NaN
188,Anaconda,Create Environment from Exising Env,conda create --name newpy39 --clone py39,NaN
189,Anaconda,Create Environment from YAML,conda env create -f BASE_REQUIREMENTS.yaml,NaN
190,Anaconda,List Available Environments,conda list,NaN


In [13]:
from st_aggrid import AgGrid, GridOptionsBuilder
import pandas as pd
import streamlit as st
import numpy as np

st.set_page_config(
    page_title="Coding Dictionary Dashboard",
    page_icon="📖",
    layout="wide",
)

# Load data from Google Sheets
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSnwd-zccEOQbpNWdItUG0qXND5rPVFbowZINjugi15TdWgqiy3A8eMRhbmSMBiRhHt1Qsry3E8tKY8/pub?output=csv'
df = pd.read_csv(url)

# Text-based filter
st.subheader("Key Word/ Phrase Search")
search_query = st.text_input("Search:", placeholder="Type to search...")

# Columns to display in the AgGrid table
visible_columns = ['Word', 'Category', 'Sub Categorization']

# Apply filter
if search_query:
    filtered_df = df[
        df.apply(
            lambda row: row.astype(str).str.contains(search_query, case=False, na=False).any(),
            axis=1,
        )
    ][visible_columns]
else:
    filtered_df = df[visible_columns]

# Configure AgGrid options
builder = GridOptionsBuilder.from_dataframe(filtered_df)
builder.configure_default_column(
    wrapText=True,  # Enable text wrapping
    autoHeight=True,  # Adjust row height automatically
    cellStyle={'textAlign': 'center'}
)

builder.configure_selection("single")  # Allow single row selection
grid_options = builder.build()

# Display the filtered table with AgGrid
st.subheader("Key Terms and Classification")
response = AgGrid(
    filtered_df,
    gridOptions=grid_options,
    height=300,
    width="100%",
    fit_columns_on_grid_load=True,
    allow_unsafe_jscode=True,  # Required for advanced JavaScript features
)

selected_rows = response.get("selected_rows", [])
try:
    # Merge selected rows with the original DataFrame
    selected_data = pd.DataFrame(selected_rows)
    if not selected_data.empty:
        # Merge the selected row and mimic transposed view
        final_df = df.merge(selected_data[['Word']], on='Word', how='inner')
        transposed_df = pd.DataFrame({
            "Field": final_df.columns,
            "Value": final_df.iloc[0]
        }).fillna("")

        transposed_df["Value"] = transposed_df.apply(
            lambda row: f"[Open Link]({row['Value']})" if row["Field"] == "Link" and row["Value"] else row['Value'],  # Return blank if value is empty
            axis=1
            )
        
        transposed_df['Value'] = np.where(transposed_df['Value']=='[Open Link](nan)',"",transposed_df['Value'])
        

        st.subheader("Key Term Reference Material")
        for _, row in transposed_df.iterrows():
            if row["Field"] == "Link":
                # Render the Link field as a markdown hyperlink
                st.markdown(f"**{row['Field']}:** {row['Value']}")
            elif row["Field"] == "Image":
                # Render the Image field as an image, handle missing or empty values
                if row["Value"] and not pd.isna(row["Value"]):  # Check if Value is valid
                    st.image(
                        row["Value"],
                        caption="Image Reference",
                        width=300
                    )
                else:
                    st.warning("No image available for this field.")  # Gracefully handle missing images
            elif row["Field"] == "Markdown":
                # Render the Markdown field as LaTeX
                st.latex(row["Value"])
            else:
                st.write(f"**{row['Field']}:** {row['Value']}")
    else:
        st.write("No row selected.")
except Exception as e:
    st.write("An error occurred:", e)

st.markdown(
    """
    ### Relevant Links
    [Raw Data](https://docs.google.com/spreadsheets/d/1tZ-_5Vv99_bm9CCEdDDN0KkmsFNcjWeKM86237yrCTQ/edit?gid=0#gid=0)
    """
)

2025-02-02 14:36:02.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.979 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.980 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:36:02.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

KeyError: "None of [Index(['Word', 'Category', 'Sub Categorization'], dtype='object')] are in the [columns]"